In [1]:
import redis
from datetime import datetime
import random

r = redis.Redis(host="redis")

In [2]:
random.seed(42)

def track(campaign_id, ad_id, action, timestamp):
    dt = datetime.fromtimestamp(timestamp)
    r.incr(f"campaigns:{campaign_id}:{action}:{dt.year}")
    r.incr(f"campaigns:{campaign_id}:{action}:{dt.year}:{dt.month}")
    r.incr(f"campaigns:{campaign_id}:{action}:{dt.year}:{dt.month}:{dt.day}")
    r.incr(f"campaigns:{campaign_id}:{action}:{dt.year}:{dt.month}:{dt.day}:{dt.hour}")
    r.incr(f"ads:{ad_id}:{action}:{dt.year}")
    r.incr(f"ads:{ad_id}:{action}:{dt.year}:{dt.month}")
    r.incr(f"ads:{ad_id}:{action}:{dt.year}:{dt.month}:{dt.day}")
    r.incr(f"ads:{ad_id}:{action}:{dt.year}:{dt.month}:{dt.day}:{dt.hour}")

def campaign_funnel(campaign, timing):
    requests = r.get(f"campaigns:{campaign}:request:{timing}")
    deliveries = r.get(f"campaigns:{campaign}:deliver:{timing}")
    views = r.get(f"campaigns:{campaign}:view:{timing}")
    clicks = r.get(f"campaigns:{campaign}:click:{timing}")
    buys = r.get(f"campaigns:{campaign}:buy:{timing}")
    return {
        "request": requests,
        "deliver": deliveries,
        "view": views,
        "click": clicks,
        "buy": buys
    }

def ad_funnel(ad, timing):
    requests = r.get(f"ads:{ad}:request:{timing}")
    deliveries = r.get(f"ads:{ad}:deliver:{timing}")
    views = r.get(f"ads:{ad}:view:{timing}")
    clicks = r.get(f"ads:{ad}:click:{timing}")
    buys = r.get(f"ads:{ad}:buy:{timing}")
    return {
        "request": requests,
        "deliver": deliveries,
        "view": views,
        "click": clicks,
        "buy": buys
    }

def campaign_funnel_for_year(campaign, year):
    return campaign_funnel(campaign, f"{year}")

def campaign_funnel_for_month(campaign, year, month):
    return campaign_funnel(campaign, f"{year}:{month}")

def campaign_funnel_for_day(campaign, year, month, day):
    return campaign_funnel(campaign, f"{year}:{month}:{day}")

def campaign_funnel_for_hour(campaign, year, month, day, hour):
    return campaign_funnel(campaign, f"{year}:{month}:{day}:{hour}")

def ad_funnel_for_year(ad, year):
    return ad_funnel(ad, f"{year}")

def ad_funnel_for_month(ad, year, month):
    return ad_funnel(ad, f"{year}:{month}")

def ad_funnel_for_day(ad, year, month, day):
    return ad_funnel(ad, f"{year}:{month}:{day}")

def ad_funnel_for_hour(ad, year, month, day, hour):
    return ad_funnel(ad, f"{year}:{month}:{day}:{hour}")

In [3]:
# clear redis
r.flushall()

for i in range(10000):
    timestamp = random.randint(1678395482, 1678395482 + 60*60*24*365)
    track("2486af7c", "ad_1", "request", timestamp)
    if random.random() < 0.9:
        track("2486af7c", "ad_1", "deliver", timestamp)
        if random.random() < 0.8:
            track("2486af7c", "ad_1", "view", timestamp)
            if random.random() < 0.7:
                track("2486af7c", "ad_1", "click", timestamp)
                if random.random() < 0.6:
                    track("2486af7c", "ad_1", "buy", timestamp)



In [4]:
print(campaign_funnel_for_year("2486af7c", 2023))
print(campaign_funnel_for_month("2486af7c", 2023, 3))
print(campaign_funnel_for_day("2486af7c", 2023, 3, 8))

print(ad_funnel_for_year("ad_1", 2023))
print(ad_funnel_for_month("ad_1", 2023, 3))
print(ad_funnel_for_day("ad_1", 2023, 3, 8))

{'request': b'8095', 'deliver': b'7287', 'view': b'5878', 'click': b'4102', 'buy': b'2470'}
{'request': b'590', 'deliver': b'538', 'view': b'431', 'click': b'313', 'buy': b'180'}
{'request': None, 'deliver': None, 'view': None, 'click': None, 'buy': None}
{'request': b'8095', 'deliver': b'7287', 'view': b'5878', 'click': b'4102', 'buy': b'2470'}
{'request': b'590', 'deliver': b'538', 'view': b'431', 'click': b'313', 'buy': b'180'}
{'request': None, 'deliver': None, 'view': None, 'click': None, 'buy': None}
